# Import Modules

In [1]:
import glob, os, time, json
import sys
sys.path.insert(0, os.path.join(os.path.expanduser('~/Research/MyRepos/'),'SensoryMotorPred'))
from utils import check_path

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from datasets import AudioDataset

import matplotlib.pyplot as plt
import argparse, random, multiprocessing
import numpy as np
import numpy as np
from pathlib import Path
import icecream as ic

import ray

ray.init(ignore_reinit_error=True)
print(f'Dashboard URL: https://{ray.get_dashboard_url()}')

2021-06-22 09:26:47,647	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


Dashboard URL: https://127.0.0.1:8265


In [80]:


savefigs=Falsedevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Trial      = 0    # Trial number
n_channels = 1
img_height = 128
img_width  = 16
stacks = (4,16) #(32, 64, 128, 256) # 
stack_sizes       = (n_channels,) + (stacks[:-1])
R_stack_sizes     = stacks
FiltSizes         = 3
Ahat_filt_sizes   = tuple([FiltSizes for _ in range(len(stack_sizes))]) # (FiltSizes, FiltSizes, FiltSizes, FiltSizes)
R_filt_sizes      = tuple([FiltSizes for _ in range(len(stack_sizes))]) # (FiltSizes, FiltSizes, FiltSizes, FiltSizes)

BatchSize   = 8
TimeSize    = 1#8
WindSize    = 16
Overlap     = 8
Nepochs     = 1500   # Number of epochs (full run through the dataset)
Tau         = 1     # Number steps to predict ##### Only implemented Tau =1 for now
output_mode = 'error' # Type of output, 'error', 'prediction', 'all' 
rootdir = Path('~/Research/PredAudio/').expanduser()
FigurePath = check_path(rootdir,'Figures')
save_path   = check_path(rootdir, 'results')
fileList    = list(save_path.glob('*params.json'))


In [81]:
########## Define Network Parameters ##########
params = {'width': img_width,
            'height': img_height,
            'BatchSize': int(BatchSize),
            'TimeSize': int(TimeSize),
            'WindSize': int(WindSize),
            'Overlap': int(Overlap),
            'Tau': int(Tau),
            'FiltNum': stack_sizes[1],
            'KSize': R_filt_sizes[0], # Filter size for Recurrent Layer
            'stack_sizes': stack_sizes,
            'R_stack_sizes': R_stack_sizes,
            'Ahat_filt_sizes': Ahat_filt_sizes,
            'R_filt_sizes': R_filt_sizes,
            'layer_loss_weightsMode': 'L_0',
            'lr': 0.005,
            'Nepochs': int(Nepochs),
            'output_mode': output_mode,
            'data_format': 'channels_first',
            'ImageSize': img_width*img_height,
            'log_freq': 10, #
            'rootdir': rootdir,
            'save_path': save_path,
            'input_shape':(BatchSize,TimeSize,img_height,img_width,1),
            'Train_paths': rootdir/'Specs_train.npy',
            'Test_paths' : rootdir/'Specs_test.npy',
            'Trial' : int(Trial),
            }

########## Input Shape for building Network ##########
if params['data_format'] == 'channels_first':
    input_shape = (params['BatchSize'], params['TimeSize'], 1, params['height'], params['width'])
else:
    input_shape = (params['BatchSize'], params['TimeSize'], params['height'], params['width'], 1)

    ########## Create Datasets and DataLoaders ##########
Dataset_Train = AudioDataset(params['Train_paths'],params['WindSize'],params['Overlap'])
Dataset_Test = AudioDataset(params['Test_paths'],params['WindSize'],params['Overlap'])
num_workers = multiprocessing.cpu_count()//2
DataLoader_Train = DataLoader(Dataset_Train, batch_size=params['BatchSize'], shuffle=True, drop_last=False, num_workers=num_workers, pin_memory=True)
DataLoader_Test = DataLoader(Dataset_Test, batch_size=params['BatchSize'], shuffle=True, drop_last=True, num_workers=num_workers, pin_memory=True)

# optimizer = torch.optim.Adam(precnet.parameters(), lr = params['lr'])
batch = next(iter(DataLoader_Train))

In [82]:
R_stack_sizes,stack_sizes

((4, 16), (1, 4))

In [93]:
class PreCNet(nn.Module):
    def __init__(self, stack_sizes, R_stack_sizes, Ahat_filt_sizes, R_filt_sizes,
                 pixel_max=1., error_activation='relu', stateful = True,
                 GRU_activation='tanh', GRU_inner_activation='hard_sigmoid',
                 output_mode='error', extrap_start_time=None, data_format = 'channels_first',
                 device='cuda',
                 lr=.003, optimizer='Adam', **kwargs):
        super(PreCNet, self).__init__()
        self.stack_sizes            = stack_sizes
        self.nb_layers              = len(stack_sizes)
        assert len(R_stack_sizes)   == self.nb_layers, 'len(R_stack_sizes) must equal len(stack_sizes)'
        self.R_stack_sizes          = R_stack_sizes
        assert len(Ahat_filt_sizes) == self.nb_layers, 'len(Ahat_filt_sizes) must equal len(stack_sizes)'
        self.Ahat_filt_sizes        = Ahat_filt_sizes
        assert len(R_filt_sizes)    == (self.nb_layers), 'len(R_filt_sizes) must equal len(stack_sizes)'
        self.R_filt_sizes           = R_filt_sizes
        self.num_layers             = len(stack_sizes)
        self.pixel_max              = pixel_max
        self.stateful               = stateful
        
        self.optimizer = optimizer
        self.lr = lr
        default_output_modes = ['prediction', 'error', 'all']
        layer_output_modes = [layer + str(n) for n in range(self.nb_layers) for layer in ['Rd', 'Ed', 'Ad', 'Ahatd','Ru', 'Eu', 'Au', 'Ahatu']]
        assert output_mode in default_output_modes + layer_output_modes, 'Invalid output_mode: ' + str(output_mode)
        self.output_mode = output_mode
        if self.output_mode in layer_output_modes:
            self.output_layer_type = self.output_mode[:-1]
            self.output_layer_num = int(self.output_mode[-1])
        else:
            self.output_layer_type = None
            self.output_layer_num = None
        self.extrap_start_time = extrap_start_time

        assert data_format in {'channels_last', 'channels_first'}, 'data_format must be in {channels_last, channels_first}'
        self.data_format = data_format
        self.channel_axis = -3 if data_format == 'channels_first' else -1
        self.row_axis = -2 if data_format == 'channels_first' else -3
        self.column_axis = -1 if data_format == 'channels_first' else -2
        self.device=device
        self.build()
        
    def build(self):
        self.layers_u = nn.ModuleDict()
        self.layers_d = nn.ModuleDict()
        self.layers_a = nn.ModuleDict()
        for l in range(nb_layers):
            nb_channels = self.R_stack_sizes[l]
            self.layers_a.add_module(str('Ahat%i' %l),nn.Conv2d(in_channels  = nb_channels, 
                                                            out_channels = self.stack_sizes[l],
                                                            stride       = (1, 1),
                                                            kernel_size  = self.Ahat_filt_sizes[l],
                                                            padding      = (-1 + self.Ahat_filt_sizes[l]) // 2))

            if l == nb_layers-1:
                nb_channels = 2 * self.stack_sizes[l] + self.R_stack_sizes[l]
            else:
                nb_channels = 2 * self.stack_sizes[l+1] + self.R_stack_sizes[l]
            self.layers_d.add_module(str('conv%i' % l),
                          nn.Conv2d(in_channels  = nb_channels, 
                                     out_channels = self.stack_sizes[l],
                                     stride       = (1, 1),
                                     kernel_size  = self.Ahat_filt_sizes[l],
                                     padding      = (-1 + self.Ahat_filt_sizes[l]) // 2))
            
            if l < nb_layers - 1:
                nb_channels = 2 * self.stack_sizes[l] + self.R_stack_sizes[l]
                self.layers_u.add_module(str('conv%i' % l),nn.Conv2d(in_channels  = nb_channels,
                                                    out_channels = self.R_stack_sizes[l],
                                                    kernel_size  = self.R_filt_sizes[l],
                                                    stride       = (1, 1),
                                                    padding      = (-1 + self.R_filt_sizes[l]) // 2)
                                                    )
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)

    def get_initial_states(self, input_shape):
        '''
        input_shape is like: (batch_size, timeSteps, Height, Width, 3)
                         or: (batch_size, timeSteps, 3, Height, Width)
        '''
        init_height = input_shape[self.row_axis]     # equal to `init_nb_rows` in original version
        init_width  = input_shape[self.column_axis]     # equal to `init_nb_cols` in original version

        base_initial_state = np.zeros(input_shape)
        non_channel_axis = -1 if self.data_format == 'channels_first' else -2
        for _ in range(2):
            base_initial_state = np.sum(base_initial_state, axis = non_channel_axis)
        base_initial_state = np.sum(base_initial_state, axis = 1)   # (batch_size, 3)

        initial_states = []
        states_to_pass = ['R', 'E']    # R is `representation`, c is Cell state in GRU, E is `error`.
        layerNum_to_pass = {sta: self.num_layers for sta in states_to_pass}
        if self.extrap_start_time is not None:
            states_to_pass.append('Ahat')   # pass prediction in states so can use as actual for t+1 when extrapolating
            layerNum_to_pass['Ahat'] = 1

        for sta in states_to_pass:
            for lay in range(layerNum_to_pass[sta]):
                downSample_factor = 2 ** lay            
                row = init_height // downSample_factor
                col = init_width  // downSample_factor
                if sta in ['R']:
                    stack_size = self.R_stack_sizes[lay]
                elif sta == 'E':
                    stack_size = self.stack_sizes[lay] * 2
                elif sta == 'Ahat':
                    stack_size = self.stack_sizes[lay]
                output_size = stack_size * row * col    # flattened size
                reducer = np.zeros((input_shape[self.channel_axis], output_size))   # (3, output_size)
                initial_state = np.dot(base_initial_state, reducer)                 # (batch_size, output_size)

                if self.data_format == 'channels_first':
                    output_shape = (-1, stack_size, row, col)
                else:
                    output_shape = (-1, row, col, stack_size)
                # initial_state = torch.from_numpy(np.reshape(initial_state, output_shape)).float().to(device)
                initial_state = Variable(torch.from_numpy(np.reshape(initial_state, output_shape)).float().to(self.device), requires_grad = True)
                initial_states += [initial_state]

        if self.extrap_start_time is not None:
            initial_states += [Variable(torch.IntTensor(1).zero_().to(self.device))]   # the last state will correspond to the current timestep

        return initial_states


    def forward(self, a, states):
        h_tm1 = states[:self.nb_layers]
        e_tm1 = states[self.nb_layers:2*self.nb_layers]
        
        if self.extrap_start_time is not None:
            t = states[-1]
        
        a0 = a[:]

        h = []
        e = []
        ahat_list=[]

        ########## Update R units starting from the top ##########
        for l in reversed(range(self.nb_layers)):
            if l == self.nb_layers - 1:
                inputs = [h_tm1[l], e_tm1[l]]
            else: 
                inputs = [h_tm1[l], ed]
                
            inputs = torch.cat(inputs, dim=self.channel_axis)
            if not isinstance(inputs, Variable):
                inputs = Variable(inputs, requires_grad=True)

            _h = hard_sigmoid(self.layers_d['conv{:d}'.format(l)](inputs))
#             z = hard_sigmoid(self.conv_layers['zd'][l](inputs))
#             o = hard_sigmoid(self.conv_layers['od'][l](inputs))
#             _o = torch.tanh(o * self.conv_layers['hd'][l](inputs))
#             _h = (1-z) * h_tm1[l] + z * _o
            h.insert(0, _h)

            ahat = self.conv_layers['ahat'][l](h[0])
            if l == 0:
                ahat[ahat > self.pixel_max] = self.pixel_max        # passed through a saturating non-linearity set at the maximum pixel value
                frame_prediction = ahat
            ahat_list.insert(0,ahat)
            
            if l > 0:
                a = self.pool(h_tm1[l-1])
            else:
                if self.extrap_start_time is not None:
                    if t >= self.t_extrap:
                        a = ahat
                    else:
                        a = a0
                else:
                    a = a0
            
            ########## compute errors ##########
            e_up = F.relu(ahat - a)
            e_down = F.relu(a - ahat)

            e.insert(0, torch.cat((e_up, e_down), dim=self.channel_axis))

            if l > 0:
                ed = self.upsample(e[0])
            
            if self.output_layer_num == l:
                if self.output_layer_type == 'Ad':
                    output = a
                elif self.output_layer_type == 'Ahatd':
                    output = ahat
                elif self.output_layer_type == 'Hd':
                    output = h[l]
                elif self.output_layer_type == 'Ed':
                    output = e[l]

        ########## Update feedforward path starting from the bottom ##########
        for l in range(self.nb_layers):
            if l == 0:
                pass
            else:
                a = self.pool(h[l-1])
                ahat = ahat_list[l]
                e_up = F.relu(ahat - a)
                e_down = F.relu(a - ahat)
                e[l] = torch.cat((e_up, e_down), axis = self.channel_axis)

            if l < self.nb_layers - 1:
                inputs = [h[l], e[l]]
                inputs = torch.cat(inputs, dim=self.channel_axis)
                if not isinstance(inputs, Variable):
                    inputs = Variable(inputs, requires_grad=True)

                z = hard_sigmoid(self.conv_layers['zu'][l](inputs))
                o = hard_sigmoid(self.conv_layers['ou'][l](inputs))
                _o = torch.tanh(o * self.conv_layers['hu'][l](inputs))
                _h = (1-z) * h[l] + z * _o
                h[l] = _h

In [94]:
model = PreCNet(params['stack_sizes'], params['R_stack_sizes'],
                    params['Ahat_filt_sizes'], params['R_filt_sizes'], 
                    pixel_max=1, output_mode=params['output_mode'], return_sequences=True,)

In [95]:
initial_state = model.get_initial_states(batch.shape)

In [96]:
initial_state[0].shape

torch.Size([8, 4, 128, 16])

In [97]:
model.layers_d['conv0'](initial_state[1].squeeze())

RuntimeError: Given groups=1, weight of size [1, 12, 3, 3], expected input[8, 16, 64, 8] to have 12 channels, but got 16 channels instead

In [98]:
layers_d,layers_u,layers_a

(ModuleList(
   (conv0): Conv2d(12, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv1): Conv2d(24, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 ),
 ModuleList(
   (conv0): Conv2d(6, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 ),
 ModuleList(
   (Ahat0): Conv2d(4, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (Ahat1): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 ))

In [67]:
batch.shape

torch.Size([8, 7, 1, 128, 16])